In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import pandas as pd
from scipy.signal import savgol_filter
import os
from sklearn import preprocessing
import pywt
from scipy.interpolate import CubicSpline
import time
from scipy.signal import detrend
from scipy.signal import find_peaks

In [ ]:
read_dir=r".."

save_dir=read_dir+"\\segmented"
if(not (os.path.exists(save_dir))):
    os.mkdir(save_dir)
csv_file=pd.read_csv(read_dir+"\\"+file_name+".csv")
fs=80

press1=csv_file.iloc[:,3]
press2=csv_file.iloc[:,4]
press1=savgol_filter(press1, window_length=11, polyorder=2)
press2=savgol_filter(press2, window_length=11, polyorder=2)

time = np.arange( 0,len(press1)/fs,1/fs)

In [ ]:
wavelet = 'db4'
level = 10
coeffs = pywt.wavedec(press2, wavelet, level=level)

coeffs[0] = np.zeros_like(coeffs[0])
press2_reconstructed = pywt.waverec(coeffs, wavelet)

coeffs = pywt.wavedec(press1, wavelet, level=level)
coeffs[0] = np.zeros_like(coeffs[0])

press1_reconstructed = pywt.waverec(coeffs, wavelet)


coeffs=pywt.wavedec(press3,wavelet,level=level)
press3_reconstructed = pywt.waverec(coeffs, wavelet)
time = np.arange( 0,len(press1_reconstructed)/fs,1/fs)

press=press1_reconstructed+press2_reconstructed

In [ ]:
def calculate_f_second_double_prime(sequence, h):
    result = []
    for index in range(len(sequence)):
        f_minus_3 = sequence[index - 3] if index >= 3 else 0
        f_minus_2 = sequence[index - 2] if index >= 2 else 0
        f_minus_1 = sequence[index - 1] if index >= 1 else 0
        f_0 = sequence[index]
        f_1 = sequence[index + 1] if index < len(sequence) - 1 else 0
        f_2 = sequence[index + 2] if index < len(sequence) - 2 else 0
        f_3 = sequence[index + 3] if index < len(sequence) - 3 else 0

        numerator = 4 * f_0 + (f_1 + f_minus_1) - 2 * (f_2 + f_minus_2) - (f_3 + f_minus_3)
        denominator = 16 * h**2
        f_second_double_prime = numerator / denominator

        result.append(f_second_double_prime)

    return result



press_acc=calculate_f_second_double_prime(press,0.0125)
window_size = 5
filtered_press = np.convolve(press, np.ones(window_size)/window_size, mode='same')

def normalize_sequence(sequence):
    max_abs = np.max(np.abs(sequence))
    normalized_sequence = sequence / max_abs
    return normalized_sequence

filtered_press=normalize_sequence(filtered_press)
filtered_press = np.convolve(press, np.ones(window_size)/window_size, mode='same')
filtered_press=normalize_sequence(filtered_press)
negative_peaks, _ = find_peaks(-filtered_press)

In [ ]:
def getTemplate(sequence):
    peak_num=len(negative_peaks)
    segmentation=[]
    for i in negative_peaks:
        s=i-39
        if(s<0):
            s=0
        e=i+40
        if(e>=len(sequence)):
            e=len(sequence)-1
        segmentation.append(sequence[s:e])


    for i in range(len(segmentation)):
        segmentation[0]+=segmentation[i]
        plt.xlim(0, 80)
        plt.plot(segmentation[0],linewidth=2)

    plt.grid(axis='x', linestyle='--')
    plt.grid(axis='y', linestyle='--')
    plt.xlabel('Sampling point', fontsize='20')
    plt.ylabel('Amplitude', fontsize='20')
    plt.savefig('segment overlaying.png', dpi=600,facecolor='white', bbox_inches='tight')
    plt.show()
    plt.plot(segmentation[0])
    plt.show()
    segmentation[0] = savgol_filter(segmentation[0], window_length=11, polyorder=2)
    acc=calculate_f_second_double_prime(segmentation[0],0.0125)


    plt.plot(acc[5:-5])
    plt.plot(segmentation[0][5:-5]*300)
    plt.show()
    peaks,_=find_peaks(acc[5:-5])
    max_peak=[]
    plt.xlim(0, 70)
    plt.plot(segmentation[0][5:-5]*300,label='result of segment overlay*300', linewidth=3)
    plt.plot(acc[5:-5], label='Signal acceleration',color= "#f51070",linewidth=3)

    for _ in range(2):
        max_peak_index = None
        max_peak_value = None
        for peak in peaks:
            if max_peak_index is None or acc[5:-5][peak] > max_peak_value:
                max_peak_index = peak
                max_peak_value = acc[5:-5][peak]

        max_peak.append(max_peak_index)
        if _ is 1:
            plt.plot(max_peak_index, max_peak_value, 'ko', markersize=10,label="peak point")
        else:
            plt.plot(max_peak_index, max_peak_value, 'ko', markersize=10)
        peaks = peaks[peaks != max_peak_index]

        plt.legend(loc='lower right', fontsize='15')

    plt.grid(axis='x', linestyle='--')
    plt.grid(axis='y', linestyle='--')
    plt.xlabel('Sampling point', fontsize='20')
    plt.ylabel('Amplitude', fontsize='20')
    print(max_peak)
    plt.savefig('accuracy peak.png', dpi=600,facecolor='white', bbox_inches='tight')
    plt.show()

    end_s=max_peak[0] if max_peak[0]>max_peak[1] else max_peak[1]
    start_s= max_peak[1] if max_peak[0]>max_peak[1] else max_peak[0]

    return segmentation[0][5:-5][start_s:end_s]/peak_num
u = getTemplate(filtered_press.copy())

In [ ]:
u=np.array(u)
def remove_duplicates_preserve_order(sequence):
    seen = set()
    unique_sequence = []
    for item in sequence:
        if item not in seen:
            seen.add(item)
            unique_sequence.append(item)
    return unique_sequence



def linear_warping(template, segment_length):
    template_length = len(template)
    template_x = np.arange(template_length)
    segment_x = np.linspace(0, template_length - 1, segment_length)
    cs = CubicSpline(template_x, template)
    warped_template = cs(segment_x)
    return warped_template


def peak_segmentation(S,u,B,peaks):
    starts=[]
    ends=[]
    bmin=B[0]
    bmax=B[1]

    for i in peaks :
        s=i-39
        if(s<0):
            s=0
        e=i+40
        if(e>=len(S)):
            e=len(S)-1
        Si=np.linalg.norm(S[s:e]-linear_warping(u,e-s),ord=1)
        Se=0
        Ss=0
        while e - s > bmin:
            Ss=np.linalg.norm(linear_warping(S[s+1:e],e-s)-linear_warping(u,e-s),ord=1)
            Se=np.linalg.norm(linear_warping(S[s:e-1],e-s)-linear_warping(u,e-s),ord=1)

            if Si<Ss and Si<Se:
                break
            if Ss>Se:
                e=e-1
                Si=Se
            else:
                s=s+1
                Si=Ss

        if(e-s<=20):
            continue
        starts.append(s)
        ends.append(e)
    return starts,ends

B=[20,80]
starts,ends=peak_segmentation(filtered_press,u,B,negative_peaks)
time = np.arange( 0,len(press2_reconstructed)/fs,1/fs)